<a href="https://colab.research.google.com/github/akash02ita/CPSC-599.6-DL-for-Vision/blob/main/a2/DL4Vision_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2
In this assignment, we will use a multi-layer perceptron network to build an image classifier for single digits. We will be using a public dataset for model development. The dataset we will be using is the MNIST digit dataset. The dataset contains 10 classes, where class `i` contains images of digit `i`.

In [20]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np


####1. Create train_data and test_dataset objects using the MNIST digit dataset from torchvision.datasets module. (5 points)

In [21]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='data', train = True, transform=transform, download = True)
test_dataset = datasets.MNIST(root='data', train = False, transform=transform, download = True)

####2. Use the [random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method to split the `train_data` into `train_dataset` (50000 images) and `validation_dataset` dataset (10000 images). (5 points)

In [22]:
print(len(train_data))
train_dataset, validation_dataset = random_split(train_data, [50000, 10000], generator=torch.Generator().manual_seed(42))
len(train_dataset), len(validation_dataset)

60000


(50000, 10000)

#### 3. Create dataloader objects for `train_dataset`, `validation_dataset`, and `test_dataset`. (5 points)

In [23]:
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle = True)
type(train_dataloader), len(train_dataloader), len(train_dataloader.dataset)

(torch.utils.data.dataloader.DataLoader, 1563, 50000)

In [35]:
for images, labels in train_dataloader:
  print(type(images), type(labels))
  print(images.shape, labels.shape)
  print(images[0].numpy().shape)
  break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([32, 1, 28, 28]) torch.Size([32])
(1, 28, 28)


#### 4. Develop an MLP model for classifying MNIST images. The developed model should have four hidden layers of 256, 128, 64, and 32 neurons. Each hidden layer should be followed with a ReLU unit and a Dropout layer (p=0.2).  (15 points)

In [38]:
flattened_image_shape = 1*28*28
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(flattened_image_shape, 256, bias=True),
    nn.ReLU(),
    nn.Linear(256, 128, bias=True),
    nn.ReLU(),
    nn.Linear(128, 64, bias=True),
    nn.ReLU(),
    nn.Linear(64, 32, bias=True),
    nn.Dropout(p=0.2),
    nn.Linear(32, 10, bias=True)
)

#### 5. Define the components needed for training a deep learning model. (10 points)



#### 6. Write the training loop and train the model for 100 epochs. Print the training and validation accuracy and loss for each epoch. (35 points)

####6. Test the model using the `test_dataset`, and report accuracy and loss. (10 points)

#### 8. We would like to see how accurate the trained model is when applied to a set of images of digits with slight differences. The image of a digit in the MNIST dataset has a black background (0 value for pixel values). This might be like writing with white chalk on a blackboard. We make slight changes in the test datasets by applying a simple change `image = 1 - image`. In other words, we invert the pixel intensity values. The resulting images resemble a digit written with a black marker on a whiteboard. Test the model using the updated test_dataset, and report your observations regarding model performance. How would you change your pipeline if you redo this experiment? (15 points)